#### Setup

In [1]:
import sys
import numpy as np
import pandas as pd
import math
import random
from concorde.tsp import TSPSolver
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
print(sys.version)

3.7.6 | packaged by conda-forge | (default, Mar 23 2020, 22:45:16) 
[Clang 9.0.1 ]


#### Traveling salesman tour

In [21]:
def solve_TSP(xlocs,ylocs):
    
    solver = TSPSolver.from_data(xlocs, ylocs, 'EUC_2D')
    solution = solver.solve()
    #obj = solution.optimal_value

    return solution

In [22]:
solution = solve_TSP(xlocs,ylocs)

In [ ]:
sluti

#### Dedicated routing strategy

In [9]:
N = 10
dmin = 0
dmax = 8

In [19]:
xlocs = 100*np.random.random(N)
ylocs = 100*np.random.random(N)
coords = [(xlocs[i],ylocs[i]) for i in range(N)]
demands = np.random.randint(dmin,dmax,N)